In 2015 the idea of creating a computer system that could recognise human was considered so outrageously challenging that it was the basis of this XKCD joke:In 2015 the idea of creating a computer system that could recognise humans was considered so outrageously challenging that it was the basis of this XKCD joke

But today, we can do exactly that, in just a few minutes, using entirely free resources!

The basic steps we'll take are:

Use DuckDuckGo to search for images of "Human Potraits"
Use DuckDuckGo to search for images of "Robots photos"
Fine-tune a pretrained neural network to recognise these two groups
Try running this model on a picture of a human and see if it works.

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai 'duckduckgo_search>=6.2'

In [ ]:
!pip install ddgs

In [ ]:
from ddgs import DDGS
from fastcore.all import *

def search_images(keywords,max_images=200):return L(DDGS().images(keywords,max_results=max_images)).itemgot('image')

In [ ]:
urls=search_images('human potrait photos',max_images=5)
urls

In [ ]:
from fastdownload import download_url
from fastai.vision.all import *
import matplotlib.pyplot as plt

In [ ]:
def show_thumbnails(urls):
    
    fig, axes = plt.subplots(1, len(urls), figsize=(15, 3))
    
    for idx, url in enumerate(urls):
        dest = f'image_{idx}.jpg'
        download_url(url, dest, show_progress=False)
        
        im = Image.open(dest)
        thumb = im.to_thumb(256, 256)
        
        axes[idx].imshow(thumb)
        axes[idx].axis('off')
        axes[idx].set_title(f'Image {idx+1}')
    
    plt.tight_layout()
    # Remove plt.show() - Jupyter will display automatically

In [ ]:
show_thumbnails(urls)

In [ ]:
urls=search_images('robots photos',max_images=5)
urls

In [ ]:
show_thumbnails(urls)

In [ ]:
searches = 'human potraits','robots'
path = Path('human_or_not')

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    time.sleep(5)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

Here what each of the DataBlock parameters means:

blocks=(ImageBlock, CategoryBlock),
The inputs to our model are images, and the outputs are categories (in this case, "human potraits" or "robots").

get_items=get_image_files, 
To find all the inputs to our model, run the get_image_files function (which returns a list of all image files in a path).

splitter=RandomSplitter(valid_pct=0.2, seed=42),
Split the data into training and validation sets randomly, using 20% of the data for the validation set.

get_y=parent_label,
The labels (y values) is the name of the parent of each file (i.e. the name of the folder they're in, which will be human potraits or tobot).

item_tfms=[Resize(192, method='squish')]
Before training, resize each image to 192x192 pixels by "squishing" it (as opposed to cropping it).

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
is_human,_,probs = learn.predict(PILImage.create('human_1.jpg'))
print(f"This is : {is_human}.")
print(f"Probability it's a human: {probs[0]:.4f}")

So, as you see, in the space of a few years, creating computer vision classification models has gone from "so hard it's a joke" to "trivially easy and free"!